In [ ]:
import geopandas as gpd
import us
import pandas as pd
import libpysal
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from helpers import (
    plot_lisa_analysis,
    plot_moran_scatter,
    calculate_moran_i_report,
    calculate_moran_local_bv,
    map_local_morans,
    calculate_moran_local,
)

This step of the tutorial focuses on downloading, cleaning, and visualizing county-level geographic data for the contiguous United States (ConUS) using FIPS codes.

- State FIPS Code: A 2-digit code representing a U.S. state or territory.
- County FIPS Code: A 3-digit code representing a county or county-equivalent within a state.

When combined, the State FIPS Code and County FIPS Code form a unique 5-digit identifier for each county in the U.S.

In [ ]:
# %% Download and clean county files
# !wget https://www2.census.gov/geo/tiger/TIGER2017/COUNTY/tl_2017_us_county.zip
# !unzip tl_2017_us_county.zip

# %% Read in county file
county = gpd.read_file("tl_2017_us_county.shp")
county.plot()

# %% Limit to ConUS
exclude_states = [
    us.states.GU.fips,
    us.states.AS.fips,
    us.states.MP.fips,
    us.states.PR.fips,
    us.states.AK.fips,
    us.states.HI.fips,
    us.states.VI.fips,
]

county = county[~county["STATEFP"].isin(exclude_states)]
county.plot()

This step processes the HMDA dataset by reading loan data, filtering it to include only approved or purchased loans, and aggregating key metrics such as total loan amount, average loan amount, and median family income. The data is grouped by state, county, loan type, and purchaser type, ensuring clean and structured data for further analysis.

*Note the use of `engine='pyarrow'`* this is a new feature in `pandas` 2.0.0 that allows for faster reading of files. If you are using an older version of `pandas`, you may need to install the `pyarrow` library separately or use the default engine.


In [ ]:
# !wget https://files.consumerfinance.gov/hmda-historic-loan-data/hmda_2017_nationwide_first-lien-owner-occupied-1-4-family-records_labels.zip
# !unzip hmda_2017_nationwide_first-lien-owner-occupied-1-4-family-records_labels.zip

# %% Read in loan data
hmda = pd.read_csv(
    "./hmda_2017_nationwide_first-lien-owner-occupied-1-4-family-records_labels.csv",
    engine="pyarrow",

)


# limit to approved or purchased loans
hmda = hmda[(hmda["action_taken"] == 1) | (hmda["action_taken"] == 6)]

# get loans by type
loans = (
    hmda.groupby(
        ["state_code", "county_code", "loan_type_name", "purchaser_type"],
        as_index=False,
    )
    .agg(
        total_loan_amount=("loan_amount_000s", "sum"),
        avg_loan_amount=("loan_amount_000s", "mean"),
        fam_income_med=("hud_median_family_income", "mean"),
    )
    .dropna()
)

loans.head()

This step standardizes FIPS codes by converting state and county codes into zero-padded strings, ensuring they follow the standard 2-digit and 3-digit formats, respectively. 

Additionally, it categorizes the `purchaser_type` variable into three groups: "not_sold," "GSE" (Government-Sponsored Enterprises), and "sold_other," using defined numeric ranges. This prepares the data for further analysis by creating consistent identifiers and meaningful categories.

Categorization of the `purchaser_type` variable is done as follows:
- 'not_sold' is (-.1,0]
- 'GSE' is (0,4]
- 'sold_other' is (4,9]

In [ ]:
# standarize FIPS codes
loans["STATEFP"] = loans["state_code"].astype(int).astype(str).str.zfill(2)
loans["COUNTYFP"] = loans["county_code"].astype(int).astype(str).str.zfill(3)

# Set purchaser type to be categorical
loans["sold"] = pd.cut(
    loans["purchaser_type"],
    bins=[-0.1, 0, 4, 9],
    labels=["not_sold", "GSE", "sold_other"],
)

This code isolates loans **sold to Government-Sponsored Enterprises (GSE)** by grouping the loans dataset based on the standardized FIPS codes (`STATEFP` and `COUNTYFP`) and the categorical `sold` column. It then aggregates key metrics for each group, including the total loan amount (`total_loan_amount`), the average loan amount (`avg_loan_amount`), and the average median family income (`fam_income_med`). 


In [ ]:
# Isolate loans sold to GSE
loans_sold = (
    loans.groupby(["STATEFP", "COUNTYFP", "sold"])
    .agg(
        total_loan_amount=("total_loan_amount", "sum"),
        avg_loan_amount=("avg_loan_amount", "mean"),
        fam_income_med=("fam_income_med", "mean"),
    )
    .reset_index()
)
loans_sold


We merge the aggregated data with the county geographic data, ensuring that the resulting dataset contains both geographic and loan-related information. This step is crucial for visualizing the data on a map, as it combines the geographic boundaries with the loan metrics.

Then it calculates the `loan_income_ratio` to assess the relationship between loan amounts and family income.  


In [ ]:

# Merge boundaries / loans
cnt_GSE_loans = pd.merge(
    county,
    loans_sold[loans_sold["sold"] == "GSE"],
    on=["STATEFP", "COUNTYFP"],
    how="left",
    validate="one_to_one",
)

# calculate loan-to-income ratio
cnt_GSE_loans["loan_income_ratio"] = (
    cnt_GSE_loans["avg_loan_amount"] / cnt_GSE_loans["fam_income_med"]
)

# plot loan-to-income ratio
ax = cnt_GSE_loans.plot("loan_income_ratio", cmap="coolwarm", legend=True)
ax.set_title("Loan-to-Income Ratio Loans Sold to GSE")


This goes through the same steps but but for **loans not sold to GSE**.  

In [ ]:
# Isolate loans not sold
cnt_notsold_loans = pd.merge(
    county,
    loans_sold[loans_sold["sold"] == "not_sold"],
    on=["STATEFP", "COUNTYFP"],
    how="left",
    validate="one_to_one",
)
cnt_notsold_loans["loan_income_ratio"] = (
    cnt_notsold_loans["avg_loan_amount"] / cnt_notsold_loans["fam_income_med"]
)
ax = cnt_notsold_loans.plot("loan_income_ratio", cmap="coolwarm", legend=True)
ax.set_title("Loan-to-Income Ratio for Not Sold Loans")


Loans not sold to GSEs are then merged into a common data set `common_loans`. The because we are mergining with more than one geometry column, `geometry_not_sold` is dropped, and the remaining geometry column is renamed and the `geometry` and projection are then reset. 

In [ ]:
# Bring loans sold and not sold together into county geodataframe

# Keep only the variables of interest
keep_variables = [
    "STATEFP",
    "COUNTYFP",
    "NAME",
    "geometry",
    "total_loan_amount",
    "avg_loan_amount",
    "fam_income_med",
    "loan_income_ratio",
]


# Merge the two dataframes
common_loans = pd.merge(
    left=cnt_GSE_loans[keep_variables].dropna(subset=["loan_income_ratio"]),
    right=cnt_notsold_loans[keep_variables].dropna(subset=["loan_income_ratio"]),
    on=["STATEFP", "COUNTYFP"],
    how="inner",
    validate="one_to_one",
    suffixes=("_GSE", "_not_sold"),
)
common_loans = common_loans.drop(columns="geometry_not_sold").rename(
    columns={"geometry_GSE": "geometry"}
)
common_loans = common_loans.set_geometry("geometry").to_crs("ESRI:102003")
common_loans.head()
# %% Drop one outlier
# common_loans = common_loans[common_loans["loan_income_ratio_not_sold"] < 0.025]

Now we can plot both variables of interest, `loan_income_ratio` and `loan_amount`, using the `.plot()` function.

In [ ]:
# %% Plot variables of interest
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

common_loans.plot(
    "loan_income_ratio_GSE",
    scheme="NaturalBreaks",
    k=5,
    edgecolor="white",
    linewidth=0.1,
    alpha=0.75,
    legend=True,
    legend_kwds={"fmt": "{:.3f}"},  # Show 4 decimal places
    ax=ax1,
)
ax1.set_title("Loan-to-Income Ratio GSE")
common_loans.plot(
    "loan_income_ratio_not_sold",
    # cmap="plasma",
    scheme="NaturalBreaks",
    k=5,
    edgecolor="white",
    linewidth=0.1,
    alpha=0.75,
    legend=True,
    legend_kwds={"fmt": "{:.3f}"},  # Show 4 decimal places
    ax=ax2,
)
ax2.set_title("Loan-to-Income Ratio not sold")

A **weights matrix** is a mathematical representation of spatial relationships between geographic entities, often used in spatial analysis. It defines how areas (e.g., counties) are connected or influence each other, typically based on adjacency or distance. In this tutorial, a **Queen contiguity weights matrix** is created using `libpysal`, where areas are considered neighbors if they share a boundary or corner. 

![alt text](images/queen_contiguity_matrix.png)

The weights are then row-standardized (`w.transform = "R"`) to ensure that the influence of each area sums to 1. This matrix is used to perform spatial autocorrelation analysis, such as calculating Moran's I, which measures the degree of spatial clustering of a variable (e.g., `loan_income_ratio`). It helps identify patterns, such as whether high or low values of the variable cluster geographically.

In [ ]:
#  Create the weights matrix from the filtered data
w = libpysal.weights.Queen.from_dataframe(common_loans, use_index=True)
w.transform = "R"  # Row-standardize the weights

In [ ]:
# show the weights
w.plot(common_loans)

> ## Moran's I Tutorial: Spatial Autocorrelation Measure
>Moran's I is a statistical measure used to assess the degree of spatial autocorrelation in a dataset. Spatial autocorrelation refers to the correlation of a variable with itself through space, indicating whether similar values cluster together geographically or are dispersed randomly.
>
> Formula:
> The formula for Moran's I is:
>
> $ I = \frac{N}{W} \cdot \frac{\sum_{i}\sum_{j}w_{ij}(x_i - \bar{x})(x_j - \bar{x})}{\sum_{i}(x_i - \bar{x})^2} $
>
> Where:
> 
> - ($N$): Total number of spatial units (e.g., counties).
> - ($W$): Sum of all spatial weights (($w_{ij}$)).
> - ($w_{ij}$): Spatial weight between units ($i$) and ($j$) (e.g., adjacency or distance-based weights).
> - ($x_i$,$ x_j$): Values of the variable of interest at locations ($i$) and ($j$).
> - ($\bar{x}$): Mean of the variable of interest.
> 
> **Interpretation:**
> - Positive Moran's I: Indicates clustering of similar values (e.g., high values near high values or low values near low values).
> - Negative Moran's I: Indicates dispersion, where dissimilar values are neighbors (e.g., high values near low values).
> - Moran's I ≈ 0: Indicates a random spatial pattern with no significant autocorrelation
> 
> In the context of the tutorial, Moran's I is used to analyze the spatial clustering of variables like loan_income_ratio. A high positive Moran's I would suggest that counties with similar loan-to-income ratios are geographically clustered, while a value near zero would indicate no spatial pattern.
> 
> ### Global Moran’s I – How it works
> Global moran's I like most measures of spatial autocorrelation leverages a **cross-product** between the distance from the mean of  location $(x_i - \bar{x})$, and the difference between the mean and the value of its neighbors $(x_j - \bar{x})$.
> 
> $(x_i - \bar{x})(x_j - \bar{x})$   <-- *this is the cross-product*
> 
> ### Cross-product Examples
> If we assume $\bar{x}$ is = 10, we can plug in some high and low values for `x_i` and `x_j` to see how the cross-product works. 
> 
> | Feature Values | Difference from Mean | Cross-product | Interpretation       | Correlation |
> |----------------|----------------------|----------------|----------------------|-------------|
> | I = 50<br>J = 40 | 40<br>30       | 1200          | High neighbor high   | +           |
> | I = 8<br>J = 6   | –2<br>–4       | 8             | Low neighbor low     | +           |
> | I = 20<br>J = 2  | –10<br>–8      | –80           | High neighbor low    | –           |
> 
> **Take aways:**
> - Positive values imply neighbors are both above or below mean
> - Negative values imply one neighbor above one below mean
> - Larger the deviation, larger the cross-product

In the following block we will calculate the global Moran's I statistic for the `loan_income_ratio` for both `GSE` and `not_sold` loans. This statistic quantifies the degree of spatial autocorrelation in the data, helping to identify whether similar values cluster together or are dispersed across the geographic area.

In [ ]:
# %% Moran's I value
# Calculate for GSE loans
mi_gse = calculate_moran_i_report(common_loans, "loan_income_ratio_GSE", w)

# Calculate for not-sold loans
mi_not_sold = calculate_moran_i_report(common_loans, "loan_income_ratio_not_sold", w)



> ## Moran’s I Scatter Plot Tutorial
> Moran’s I is a global measure of spatial autocorrelation — it assesses whether similar attribute values cluster spatially. The **Moran’s I scatter plot** is a graphical diagnostic tool that helps visualize these spatial relationships.
> 
> ### Plot Overview
> 
> This example shows **GSE (Government-Sponsored Enterprise) loans** across spatial units.
> 
> - **X-axis (Attribute):** Standardized value of the variable at each location (e.g., proportion of GSE loans).
> - **Y-axis (Spatial Lag):** Spatially lagged value — the weighted average of neighboring values.
> - **Each Point:** Represents a spatial unit.
> - **Black Trend Line:** Indicates the degree and direction of spatial autocorrelation (the slope is the Moran’s I value).
> 
> ### Interpretation by Quadrant
> 
> The scatter plot is divided into four quadrants:
> 
> | Quadrant        | Description                     | Spatial Pattern     |
> |-----------------|----------------------------------|---------------------|
> | **Upper Right** | High values surrounded by high  | High–High (Red Cluster) |
> | **Lower Left**  | Low values surrounded by low    | Low–Low (Blue Cluster)   |
> | **Upper Left**  | Low values surrounded by high   | Low–High (Orange Outlier)  |
> | **Lower Right** | High values surrounded by low   | High–Low (Light Blue Outlier)  |
> 
> Points in the upper right and lower left indicate **positive spatial autocorrelation** (clustering), while points in the other two quadrants indicate **spatial outliers**.
> 
> ### Take aways:
> 
> - **Positive Moran’s I**: Similar values cluster (as seen in this plot).
> - **Negative Moran’s I**: Dissimilar values are neighbors (more points in opposing quadrants).
> - The **slope** of the trend line is a visual approximation of the **Moran’s I statistic**, which ranges from –1 (perfect dispersion) to +1 (perfect clustering).

In [ ]:
#  Moran's Scatter
plot_moran_scatter(df=common_loans,
    x_name="loan_income_ratio_GSE", w=w , title="GSE loans"
)

In [ ]:
plot_moran_scatter(
    df=common_loans, x_name="loan_income_ratio_not_sold", w=w, title="not sold"
)

In [ ]:

# %%
# from splot.esda import plot_local_autocorrelation, moran_scatterplot
# from esda.moran import Moran_Local

# plot_local_autocorrelation(
#     moran_loc=Moran_Local(common_loans["loan_income_ratio_GSE"].values, w),
#     gdf=common_loans,
#     attribute="loan_income_ratio_GSE",
# )


In [ ]:

# %%
moran_scatterplot(Moran_Local(common_loans["loan_income_ratio_GSE"].values, w), p=0.05)


In [ ]:

# %%
from esda.moran import Moran_Local_BV

moran_scatterplot(
    Moran_Local_BV(
        common_loans["loan_income_ratio_GSE"].values,
        common_loans["loan_income_ratio_not_sold"].values,
        w,
    ),
    p=0.05,
)


In [ ]:

# %% interactive local morans bivariate

quadrant_GSEvNotSold = calculate_moran_local_bv(
    gdf=common_loans,
    x_name="loan_income_ratio_GSE",
    y_name="loan_income_ratio_not_sold",
    w=w,
)


In [ ]:
# %%
color_mapping = {
    "HH": "red",
    "HL": "orange",
    "LH": "lightblue",
    "LL": "blue",
    "NS": "lightgrey",
}
quadrant_GSEvNotSold["color"] = quadrant_GSEvNotSold["quadrant_label"].map(color_mapping)

# Use the mapped colors in the explore function
quadrant_GSEvNotSold.explore(
    column="quadrant_label",
    cmap=list(color_mapping.values()),  # Ensure the colors match the legend order
    tooltip=False,
    popup=True,
    legend=True,  # Enable legend
    legend_kwds={"colorbar": False, "caption": "Quadrant"},
)

In [ ]:
map_local_morans(
    gdf=common_loans,
    x_name="loan_income_ratio_GSE",
    y_name="loan_income_ratio_not_sold",
    w=w,
)

In [ ]:
quadrant_GSEvNotSold = calculate_moran_local_bv(
    gdf=common_loans,
    x_name="loan_income_ratio_GSE",
    y_name="loan_income_ratio_not_sold",
    w=w,
)

In [ ]:

# %%
# %% Lisa PLots - single variable
plot_lisa_analysis(
    df=common_loans,
    x_name="loan_income_ratio_GSE",
    w=w,
    title_prefix="loan_income_ratio_GSE",
    legend_kwds={"fmt": "{:.4f}"},
)


In [ ]:

# %%
plot_lisa_analysis(
    df=common_loans,
    x_name="loan_income_ratio_not_sold",
    w=w,
    title_prefix="",
    legend_kwds={"fmt": "{:.4f}"},
)


In [ ]:


# %% Lisa Plot - multivariable

plot_lisa_analysis(
    df=common_loans,
    x_name="loan_income_ratio_GSE",
    y_name="loan_income_ratio_not_sold",
    w=w,
    title_prefix="GSE/nosold",
    legend_kwds={"fmt": "{:.4f}"},
)